In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import cv2
import mediapipe as mp
import time
import pyttsx3
## Url data from Kaggle >>> https://www.kaggle.com/datasets/grassknoted/asl-alphabet/code
data_dir = '/Users/albaqoum/Downloads'

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

print(f"Number of Classes: {train_data.num_classes}")
print("Classes Name :", list(train_data.class_indices.keys()))

def create_and_train_model():
    num_classes = train_data.num_classes 

    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, validation_data=val_data, epochs=10)
    model.save("asl_new_model.h5")
    return model, history

def load_or_create_model():
    if os.path.exists("asl_model.h5"):
        print('Download trainer later')
        return load_model("asl_model.h5"), None
    else:
        print("Create and train a new model")
        return create_and_train_model()

def setup_tts_engine():
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)
    engine.setProperty('volume', 0.9)
    return engine

class HandDetector:
    def __init__(self, mode=False, max_hands=1, detection_confidence=0.5, tracking_confidence=0.5):
        self.mode = mode
        self.max_hands = max_hands
        self.detection_confidence = detection_confidence
        self.tracking_confidence = tracking_confidence

        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=self.mode,
            max_num_hands=self.max_hands,
            min_detection_confidence=self.detection_confidence,
            min_tracking_confidence=self.tracking_confidence
        )
        self.mp_draw = mp.solutions.drawing_utils

    def find_hands(self, img, draw=True):
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).copy()
        self.results = self.hands.process(img_rgb)

        if self.results.multi_hand_landmarks:
            for hand_landmarks in self.results.multi_hand_landmarks:
                if draw:
                    self.mp_draw.draw_landmarks(img, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)
        return img

    def get_hand_bbox(self, img):
        bbox = None
        if self.results.multi_hand_landmarks:
            for hand_landmarks in self.results.multi_hand_landmarks:
                x_coordinates = [lm.x for lm in hand_landmarks.landmark]
                y_coordinates = [lm.y for lm in hand_landmarks.landmark]

                min_x, max_x = min(x_coordinates), max(x_coordinates)
                min_y, max_y = min(y_coordinates), max(y_coordinates)

                h, w, c = img.shape
                min_x, max_x = int(min_x * w), int(max_x * w)
                min_y, max_y = int(min_y * h), int(max_y * h)

                padding = 20
                min_x = max(0, min_x - padding)
                min_y = max(0, min_y - padding)
                max_x = min(w, max_x + padding)
                max_y = min(h, max_y + padding)

                bbox = (min_x, min_y, max_x, max_y)
        return bbox

def preprocess_hand_image(hand_img):
    resized_img = cv2.resize(hand_img, (64, 64))
    normalized_img = resized_img / 255.0
    return np.expand_dims(normalized_img, axis=0)

def get_class_name(index, class_indices):
    class_names = {v: k for k, v in class_indices.items()}
    return class_names[index]

def main():
    model, _ = load_or_create_model()

    detector = HandDetector(detection_confidence=0.7)
    tts_engine = setup_tts_engine()

    cap = cv2.VideoCapture(0)
    last_prediction_time = 0
    prediction_interval = 2 
    last_prediction = ""

    print('Exit enter q')
    while True:
        success, img = cap.read()
        if not success:
            print("Failed to read frame from camera")
            break

        img = detector.find_hands(img)
        bbox = detector.get_hand_bbox(img)

        if bbox:
            min_x, min_y, max_x, max_y = bbox
            cv2.rectangle(img, (min_x, min_y), (max_x, max_y), (0, 255, 0), 2)
            hand_img = img[min_y:max_y, min_x:max_x]

            current_time = time.time()
            if current_time - last_prediction_time > prediction_interval and hand_img.size > 0:
                processed_img = preprocess_hand_image(hand_img)
                prediction = model.predict(processed_img)
                predicted_class_index = np.argmax(prediction)
                predicted_class = get_class_name(predicted_class_index, train_data.class_indices)
                confidence = prediction[0][predicted_class_index]

                if predicted_class != last_prediction and confidence > 0.7:
                    last_prediction = predicted_class
                    last_prediction_time = current_time
                    print(f"Deacted  {predicted_class},  {confidence:.2f}")
                    tts_engine.say(f"Letter {predicted_class}")
                    tts_engine.runAndWait()

                label = f"{predicted_class}: {confidence:.2f}"
                cv2.putText(img, label, (min_x, min_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        cv2.imshow("ASL Recognition", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
